In [21]:
from utils.em_utilities import *
import gensim
import copy
from gensim.parsing.preprocessing import STOPWORDS
import json

In [5]:
from nltk.tokenize import word_tokenize
import re

In [12]:
# def translate_non_alphanumerics(to_translate, translate_to=u'_'):
#     not_letters_or_digits = u'!"%\'()&*+,-/:;<=>?[\]^_`{|}~'
#     if isinstance(to_translate, unicode):
#         translate_table = dict((ord(char), unicode(translate_to))
#                                for char in not_letters_or_digits)
#     else:
#         assert isinstance(to_translate, str)
#         translate_table = string.maketrans(not_letters_or_digits,
#                                            translate_to
#                                               *len(not_letters_or_digits))
#     return to_translate.translate(translate_table)

def remove_username(tweet):
    p = re.compile(ur'(@[A-Za-z0-9_-]+)', re.MULTILINE | re.IGNORECASE)
    return re.sub(p, ' ', tweet)
def remove_hashtag(tweet):
    p = re.compile(ur'(#[A-Za-z0-9_-]+)', re.MULTILINE | re.IGNORECASE)
    return re.sub(p, ' ', tweet)

def preprocess(tweet):
    tweet = tweet.lower()
    tweet = remove_username(tweet)
    tweet = remove_hashtag(tweet)
    tweet = re.sub(r"https?:\/\/\S+\b|www\.(\w+\.)+\S*", " ", tweet)
    tweet = re.sub(ur"\s+"," ", tweet)
    return tweet.split()
    

In [8]:
data = json.load(open('../Data/gnip_sample2m.json'))
print "'The length of data is {}".format(len(data))
### Extract the twitter text from the lists of data
text = [item[u'twitter_text'] for item in data]

'The length of data is 2027707


In [14]:
twitter_tokens = []
for text_ in text:
    twitter_tokens.append(preprocess(text_))

In [19]:
%time twitter_dictionary = gensim.corpora.Dictionary(twitter_tokens)
print(twitter_dictionary)

CPU times: user 45.3 s, sys: 136 ms, total: 45.5 s
Wall time: 45.2 s
Dictionary(592713 unique tokens: [u'yall!!!...', u'"humble,', u'appear?williams:', u'sowell', u'done????"']...)


In [22]:
twitter_dictionary_copy = copy.copy(twitter_dictionary)

In [23]:
# ignore words that appear in less than 20 documents or more than 10% documents
twitter_dictionary_copy.filter_extremes(no_below=3, no_above=0.1)
print(twitter_dictionary_copy)

Dictionary(100000 unique tokens: [u'fawn', u'considered.', u'ready!!', u'1,800', u'(photos)']...)


In [24]:
twitter_corpus = [twitter_dictionary_copy.doc2bow(tokens) for tokens in twitter_tokens]

In [25]:
%time tfidf_model = gensim.models.TfidfModel(twitter_corpus , id2word=twitter_dictionary_copy)

CPU times: user 5.08 s, sys: 40 ms, total: 5.12 s
Wall time: 5.09 s


In [118]:
cPickle.dump(twitter_dictionary_copy , open('../Data/twitter_dictionary_gensim.pkl' , 'wb') , protocol=2)

In [28]:
%time lda_model = gensim.models.LdaModel(tfidf_model[twitter_corpus], num_topics=101, id2word=twitter_dictionary_copy, passes=5)

CPU times: user 2h 59min 27s, sys: 28min 18s, total: 3h 27min 46s
Wall time: 3h 27min 13s


In [29]:
lda_model.save("../lda_2million_gensim.bin")

In [30]:
import cPickle
cPickle.dump(lda_model , open("lda_2m_twitter.pkl" , "wb") , protocol = 2)

In [44]:
def get_test_topic(tweet):
    tokens = preprocess(tweet)
    test_corpus = twitter_dictionary_copy.doc2bow(tokens)
    test_tfidf = tfidf_model[[test_corpus]]
    return test_tfidf

In [116]:
lda_model[get_test_topic('awesome movies')]

[(83, 0.33663366336633654), (99, 0.3366336633663366)]

In [117]:
res_distrib = best_distrib_of_terms('movies')
sorted(res_distrib.items() , key = lambda x: x[1] , reverse=True)

[(99, 0.0029312064462636706),
 (92, 2.9832146846089092e-07),
 (39, 2.5832053891970964e-07),
 (61, 2.5685670361347355e-07),
 (25, 2.3712517563663293e-07),
 (27, 2.3607899850512241e-07),
 (11, 2.3344048315740348e-07),
 (91, 2.3036061962474435e-07),
 (12, 2.2626848045637012e-07),
 (36, 2.2367218278911727e-07),
 (15, 2.2130680051469629e-07),
 (80, 2.2099572502742874e-07),
 (24, 2.1981624168769979e-07),
 (82, 2.176891995047032e-07),
 (78, 2.1695992571887797e-07),
 (81, 2.1646164998673258e-07),
 (77, 2.0943667202366531e-07),
 (89, 2.0912379918049383e-07),
 (14, 2.0527288323177404e-07),
 (42, 2.0434113417361751e-07),
 (21, 2.0412234188188031e-07),
 (26, 2.0160814255742915e-07),
 (46, 2.0140406630825352e-07),
 (17, 2.0090501889811041e-07),
 (86, 1.998874928313648e-07),
 (47, 1.9907840845331973e-07),
 (18, 1.9848919868372351e-07),
 (28, 1.9650454707204981e-07),
 (59, 1.9581432239827866e-07),
 (6, 1.9577544534553154e-07),
 (96, 1.9548335276298048e-07),
 (76, 1.9496495944323695e-07),
 (84, 1.9222

In [101]:
def best_distrib_of_terms(term):

    topic_distrib_dict = {}
    term_id = twitter_dictionary_copy.token2id[term]
    for i in xrange(lda_model.num_topics):
        term_distrib = lda_model.get_topic_terms(i, topn=100000)
        for a, b in term_distrib:
            if a == term_id:
                topic_distrib_dict[i] = b
                break
    return topic_distrib_dict

    


In [52]:
import pyLDAvis.gensim as gensimvis
import pyLDAvis

In [38]:
vis_data = gensimvis.prepare(lda_model, twitter_corpus, twitter_dictionary_copy)
pyLDAvis.display(vis_data)

In [59]:
from collections import Counter

In [61]:
all_tokens = []
for i in twitter_tokens:
    all_tokens.extend(i)
word_count_dict = Counter(all_tokens)

In [62]:
word_count = sorted(word_count_dict.items() , key = lambda x: x[1] , reverse=True)

In [63]:
def give_topN(word_count_ , topn = 20):
    
    return word_count_[:topn]

In [82]:
'i' in twitter_dictionary_copy.values()

True

In [75]:
arabic =  '''random Arabic lorem ipsum copy. ... Randomly generated typographic filler text [Arabic]. HTML Plain text. الله الشّعبين أم بلا, يونيو فاتّبع العالم عن كلّ.'''

In [76]:
arabic.split()

['random',
 'Arabic',
 'lorem',
 'ipsum',
 'copy.',
 '...',
 'Randomly',
 'generated',
 'typographic',
 'filler',
 'text',
 '[Arabic].',
 'HTML',
 'Plain',
 'text.',
 '\xd8\xa7\xd9\x84\xd9\x84\xd9\x87',
 '\xd8\xa7\xd9\x84\xd8\xb4\xd9\x91\xd8\xb9\xd8\xa8\xd9\x8a\xd9\x86',
 '\xd8\xa3\xd9\x85',
 '\xd8\xa8\xd9\x84\xd8\xa7,',
 '\xd9\x8a\xd9\x88\xd9\x86\xd9\x8a\xd9\x88',
 '\xd9\x81\xd8\xa7\xd8\xaa\xd9\x91\xd8\xa8\xd8\xb9',
 '\xd8\xa7\xd9\x84\xd8\xb9\xd8\xa7\xd9\x84\xd9\x85',
 '\xd8\xb9\xd9\x86',
 '\xd9\x83\xd9\x84\xd9\x91.']

In [79]:
print  '\xd8\xa7\xd9\x84\xd9\x84\xd9\x87'.decode('utf-8')

الله


In [80]:
'\xd8\xa7\xd9\x84\xd9\x84\xd9\x87'.decode('utf-8')

u'\u0627\u0644\u0644\u0647'